In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd.profiler import record_function

In [2]:
import sys
sys.path.insert(1, "../../../")

In [3]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.yolov8cls_path import Model
from evaluation import count_parameters, show_tree, generate_dataframe

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 
device=torch.device('cpu') 

In [5]:
data_path = '../../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [6]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


# Inference at 640x640

In [126]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((640, 640)),
                                      transforms.Normalize(mean=means, std=stds)])

In [127]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [128]:
from torch.profiler import profile, ProfilerActivity

In [129]:
model = Model(num_classes=1000, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [130]:
model.eval()
model.training

False

In [131]:
next(model.parameters()).device

device(type='cpu')

In [90]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(12):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total MFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.05%     543.000us       100.00%        1.179s     117.857ms

In [92]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(3):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.13%     158.300us       100.00%     122.137ms     122.137ms

In [66]:
df = generate_dataframe(prof)

/mnt/c/Me/PJAIT/Thesis/Code/yolov2-to-yolov8/Backbone experiments/YOLOv8s-cls Path/Version 5/../../../evaluation.py:27: FutureWarning: `cuda_time` is deprecated, please use `device_time` instead.
  row[attr] = getattr(obj, attr)
/mnt/c/Me/PJAIT/Thesis/Code/yolov2-to-yolov8/Backbone experiments/YOLOv8s-cls Path/Version 5/../../../evaluation.py:27: FutureWarning: `cuda_time_total` is deprecated. Use `device_time_total` instead.
  row[attr] = getattr(obj, attr)
/mnt/c/Me/PJAIT/Thesis/Code/yolov2-to-yolov8/Backbone experiments/YOLOv8s-cls Path/Version 5/../../../evaluation.py:27: FutureWarning: `self_cuda_memory_usage` is deprecated. Use `self_device_memory_usage` instead.
  row[attr] = getattr(obj, attr)
/mnt/c/Me/PJAIT/Thesis/Code/yolov2-to-yolov8/Backbone experiments/YOLOv8s-cls Path/Version 5/../../../evaluation.py:27: FutureWarning: `self_cuda_time_total` is deprecated. Use `self_device_time_total` instead.
  row[attr] = getattr(obj, attr)


# GFLOPs at 640x640 using Ultralytics functions

In [135]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [136]:
get_flops(model, imgsz=640)

13.609779199999998

In [137]:
get_flops_with_torch_profiler(model, imgsz=640)

12.7034368

# GFLOPs at 224x224 using Ultralytics functions

In [138]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [139]:
get_flops(model, imgsz=224)

1.667197952

In [141]:
get_flops_with_torch_profiler(model, imgsz=224)

1.5561710080000002

# Inference at 224x224

In [7]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [8]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [9]:
from torch.profiler import profile, ProfilerActivity

In [10]:
model = Model(num_classes=1000, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [24]:
img = train_dataset[6][0].unsqueeze(0).to(device)
img.shape

torch.Size([1, 3, 224, 224])

In [25]:
model.eval()
model.training

False

In [26]:
next(model.parameters()).device

device(type='cpu')

In [33]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(12):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.16%     400.200us       100.00%     254.617ms      25.462ms           0 b           0 b            10                                                                                []            --  
                       infer

In [34]:
img.shape, img.device

(torch.Size([1, 3, 224, 224]), device(type='cpu'))

In [42]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(3):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.29%      72.500us       100.00%      25.368ms      25.368ms           0 b           0 b             1                                                                                []            --  
                       infer

# Count Parameters

My implementation:

In [26]:
count_parameters(model)

6361736

Ultralytics implementation:

In [27]:
model = torch.load('../../../Models/yolov8s-cls.pt')['model'].model

In [28]:
count_parameters(model)

6361736